In [1]:
import pandas as pd

# Read the four CSV files into Pandas DataFrames
df_f1_part1 = pd.read_csv('f1_part1.csv')
df_f1_part2 = pd.read_csv('f1_part2.csv')
df_f2_part1 = pd.read_csv('f2_part1.csv')
df_f2_part2 = pd.read_csv('f2_part2.csv')

# Concatenate the DataFrames vertically
combined_df = pd.concat([df_f1_part1, df_f1_part2, df_f2_part1, df_f2_part2], ignore_index=True)

# Display the columns of the combined DataFrame
print("Columns of the combined DataFrame:")
print(combined_df.columns)

Columns of the combined DataFrame:
Index(['timestamp_call_key', 'retailer_code', 'serial', 'reason', 'mos',
       'resolved', 'no_of_accounts_with_syf_13_march',
       'account_balance_13_march', 'delinquency_history_13_march',
       'account_open_date_13_march', 'account_status_13_march',
       'card_activation_status_13_march', 'eservice_ind_13_march',
       'ebill_enrolled_status_13_march', 'auto_pay_enrolled_status_13_march',
       'no_of_accounts_with_syf_18_march', 'account_balance_18_march',
       'delinquency_history_18_march', 'account_open_date_18_march',
       'account_status_18_march', 'card_activation_status_18_march',
       'eservice_ind_18_march', 'ebill_enrolled_status_18_march',
       'auto_pay_enrolled_status_18_march'],
      dtype='object')


In [2]:
combined_df.head()

,timestamp_call_key,retailer_code,serial,reason,mos,resolved,no_of_accounts_with_syf_13_march,account_balance_13_march,delinquency_history_13_march,account_open_date_13_march,...,auto_pay_enrolled_status_13_march,no_of_accounts_with_syf_18_march,account_balance_18_march,delinquency_history_18_march,account_open_date_18_march,account_status_18_march,card_activation_status_18_march,eservice_ind_18_march,ebill_enrolled_status_18_march,auto_pay_enrolled_status_18_march
0,240315173250 DAL1OA3M,D,10010009499,PP,IA PP TR,resolved,1.0,NaN,[00],5/19/2013,...,0,1.0,NaN,[00],5/19/2013,NaN,0,1,E,0
1,240315124742 DAL15XHK,E,10006021998,BA,IA BA TS nl TR,floor,2.0,143.28,[00],9/22/2003,...,0,2.0,143.28,[00],9/22/2003,NaN,0,1,E,0
2,240317114237 DAL3BLK6,J,10016007830,PT,IA AA BA MR nl PT TR,resolved,12.0,0.00,[00],9/4/2017,...,0,12.0,0.00,[00],9/4/2017,NaN,8,0,,0
3,240314120849 PHX6PL4U,E,10008003199,BA,IA BA,resolved,2.0,44.98,[00],2/7/2024,...,0,2.0,-6.32,[00],2/7/2024,NaN,0,0,L,0
4,240314145902 DAL68RYS,I,10013002037,BA,IA BA TS nl mm TR,floor,6.0,7.00,[00],1/2/2022,...,0,6.0,7.00,[00],1/2/2022,NaN,0,1,,0


In [3]:
duplicated_serials = combined_df[combined_df['serial'].duplicated(keep=False)]
duplicated_serials

,timestamp_call_key,retailer_code,serial,reason,mos,resolved,no_of_accounts_with_syf_13_march,account_balance_13_march,delinquency_history_13_march,account_open_date_13_march,...,auto_pay_enrolled_status_13_march,no_of_accounts_with_syf_18_march,account_balance_18_march,delinquency_history_18_march,account_open_date_18_march,account_status_18_march,card_activation_status_18_march,eservice_ind_18_march,ebill_enrolled_status_18_march,auto_pay_enrolled_status_18_march
4,240314145902 DAL68RYS,I,10013002037,BA,IA BA TS nl mm TR,floor,6.0,7.00,[00],1/2/2022,...,0,6.0,7.00,[00],1/2/2022,NaN,0,1,,0
5,240314151323 PHX4I30S,I,10013002037,PS,IA BA TS nl PS nl mt RS TR,floor,6.0,7.00,[00],1/2/2022,...,0,6.0,7.00,[00],1/2/2022,NaN,0,1,,0
7,240316171347 PHX4I164,B,10002094529,BA,IA BA,resolved,6.0,0.00,[00],9/13/2020,...,0,6.0,0.00,[00],9/13/2020,NaN,0,0,,0
8,240316171945 DAL16OYM,B,10002094529,BA,IA IA IA BA,resolved,6.0,0.00,[00],9/13/2020,...,0,6.0,0.00,[00],9/13/2020,NaN,0,0,,0
17,240314081235 DAL1ROPP,I,10013000316,BA,IA AA BA MR nl TR,floor,2.0,8.37,[00],2/5/2024,...,0,2.0,0.00,[00],2/5/2024,F,7,0,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1798787,240315101845 PHX3VAJ9,f,20006017345,PP,IA PP TR,resolved,3.0,479.33,[00],8/16/2023,...,0,3.0,479.33,[00],8/16/2023,NaN,0,0,,0
1798791,240314075300 PHX5SRLR,b,20002002187,PP,mn IA PP TR,resolved,9.0,680.86,[00],12/19/2023,...,0,9.0,680.86,[00],12/19/2023,NaN,0,0,NaN,0
1798792,240314080117 DAL3CVDQ,b,20002002187,BA,mn IA PP BA MR nl WA nl BA nl mt,floor,9.0,680.86,[00],12/19/2023,...,0,9.0,680.86,[00],12/19/2023,NaN,0,0,NaN,0
1798793,240314120118 DAL2JPAQ,i,20012001463,BA,IA BA TA nl,resolved,14.0,1314.94,[00],2/8/2022,...,0,14.0,1314.94,[00],2/8/2022,NaN,0,0,,0


In [4]:
value_counts = duplicated_serials['serial'].value_counts()
max_recalls = value_counts.max()
max_recalls

47

In [10]:
values_with_max_recalls = value_counts[value_counts == max_recalls].index
rows_with_max_recalls = duplicated_serials[duplicated_serials['serial'].isin(values_with_max_recalls)]
rows_with_max_recalls

,timestamp_call_key,retailer_code,serial,reason,mos,resolved,no_of_accounts_with_syf_13_march,account_balance_13_march,delinquency_history_13_march,account_open_date_13_march,...,auto_pay_enrolled_status_13_march,no_of_accounts_with_syf_18_march,account_balance_18_march,delinquency_history_18_march,account_open_date_18_march,account_status_18_march,card_activation_status_18_march,eservice_ind_18_march,ebill_enrolled_status_18_march,auto_pay_enrolled_status_18_march
260237,240314015025 PHX7K2IX,D,10023000027,AT,IA BA MR nl AT,resolved,7.0,NaN,[00],8/12/2019,...,0,7.0,NaN,[00],8/12/2019,NaN,0,0,,0
260238,240314092243 DAL5BNPX,D,10023000027,AT,IA BA MR nl AT,resolved,7.0,NaN,[00],8/12/2019,...,0,7.0,NaN,[00],8/12/2019,NaN,0,0,,0
260239,240314101732 DAL3OMIM,D,10023000027,BA,IA BA,resolved,7.0,NaN,[00],8/12/2019,...,0,7.0,NaN,[00],8/12/2019,NaN,0,0,,0
260240,240314101858 DAL5TLUG,D,10023000027,AT,IA BA MR nl AT,resolved,7.0,NaN,[00],8/12/2019,...,0,7.0,NaN,[00],8/12/2019,NaN,0,0,,0
260241,240314125532 PHX3HDFE,D,10023000027,CT,IA BA MR nl AT nl CT mt RS TR,floor,7.0,NaN,[00],8/12/2019,...,0,7.0,NaN,[00],8/12/2019,NaN,0,0,,0
260242,240314162029 PHX3KJNH,D,10023000027,AT,IA BA MR nl AT,resolved,7.0,NaN,[00],8/12/2019,...,0,7.0,NaN,[00],8/12/2019,NaN,0,0,,0
260243,240314180025 PHX3YJW9,D,10023000027,AT,IA BA MR nl AT,resolved,7.0,NaN,[00],8/12/2019,...,0,7.0,NaN,[00],8/12/2019,NaN,0,0,,0
260244,240314184430 PHX6RD93,D,10023000027,BA,IA BA,resolved,7.0,NaN,[00],8/12/2019,...,0,7.0,NaN,[00],8/12/2019,NaN,0,0,,0
260245,240314224607 DAL18VQR,D,10023000027,BA,IA BA,resolved,7.0,NaN,[00],8/12/2019,...,0,7.0,NaN,[00],8/12/2019,NaN,0,0,,0
260246,240315003551 DAL6BBOV,D,10023000027,BA,IA BA,resolved,7.0,NaN,[00],8/12/2019,...,0,7.0,NaN,[00],8/12/2019,NaN,0,0,,0


In [6]:
num_unique_recalls = duplicated_serials["serial"].nunique()
num_unique_recalls

214468

In [7]:
len(combined_df) - combined_df["serial"].nunique()

290795

In [8]:
combined_df["reason"].nunique()

73

In [9]:
from scipy.stats import chi2_contingency
contingency_table = pd.crosstab(combined_df['reason'], combined_df['account_balance_18_march'] - combined_df['account_balance_13_march'])

chi2, p, dof, expected = chi2_contingency(contingency_table)
print("P-value:", p)
print(chi2)

P-value: 0.0
9438735.439465221
